In [2]:
import sys
sys.path.append("../src")

%load_ext autoreload
%autoreload 2

from mlleakcatcher import MlLeakCatcher

In [3]:
import numpy as np
import pandas as pd

# Data preparation


In [16]:
def get_dataset_with_target_leakage(
    data_size,
    n_features) -> pd.DataFrame:


    data = pd.DataFrame(np.random.rand(data_size, n_features - 1), columns=[f'feature{i}' for i in range(1, n_features)])

    data['target'] = np.random.choice([0, 1], size=data_size)

    data['index'] = np.arange(data_size)
    data['dt'] = pd.date_range(
        start='2021-01-01', periods=data_size, freq='D')

    return data


data = get_dataset_with_target_leakage(data_size=1000,n_features=100)
data_size = 1000
data['feature100'] = data['target'] + np.random.normal(0, 0.1, size=data_size)
data['feature101'] = data['target'] + np.random.normal(0, 0.2, size=data_size)
data['feature102'] = data['target'] + np.random.normal(0, 0.3, size=data_size)
data.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature96,feature97,feature98,feature99,target,index,dt,feature100,feature101,feature102
0,0.744845,0.655613,0.417751,0.795136,0.042528,0.940251,0.202801,0.532234,0.901881,0.834832,...,0.400583,0.482433,0.986439,0.858954,0,0,2021-01-01,0.177121,0.078091,0.357197
1,0.527662,0.563399,0.353239,0.668745,0.531727,0.074906,0.543342,0.636417,0.529613,0.176369,...,0.488030,0.051004,0.429287,0.614975,1,1,2021-01-02,1.016454,1.127757,1.481938
2,0.093283,0.600842,0.471484,0.050616,0.032097,0.762769,0.787179,0.532144,0.945017,0.967028,...,0.705061,0.561345,0.583966,0.787270,0,2,2021-01-03,-0.030200,0.225320,-0.030501
3,0.362986,0.328648,0.115159,0.262286,0.554322,0.628928,0.629916,0.716186,0.873300,0.927992,...,0.922426,0.141853,0.170101,0.703822,1,3,2021-01-04,1.089978,0.800008,0.536730
4,0.010791,0.355235,0.638881,0.695290,0.364583,0.507179,0.928129,0.614262,0.324403,0.175159,...,0.536893,0.420014,0.292214,0.729012,0,4,2021-01-05,-0.108136,-0.003675,-0.452636


In [5]:
mlc = MlLeakCatcher(task_type="classification")

Logging level set to WARNING (verbosity=0)


In [13]:
features, results = mlc.run(train_data=data, target_col='target', id_cols = ['index',"dt"])

Test dataset is required for feature_target_pps_change
Check 'model_target_check' not found in registry


In [14]:
results

[
 CheckResult:
   Name: identifiertargetpps
   Deleted Features: None
   Warning Features: None
   Result DataFrame (Preview):
   Identifier  Predictive Power Score (PPS)
        index                             0
           dt                             0
   Deletion Reasons:
   None,
 
 CheckResult:
   Name: featuretargetpps
   Deleted Features: feature100, feature101, feature102
   Warning Features: None
   Result DataFrame (Preview):
    Feature  Predictive Power Score (PPS)
   feature1                           0.0
   feature2                           0.0
   feature3                           0.0
   Deletion Reasons:
   feature100: High PPS: 1.000
   feature101: High PPS: 0.973
   feature102: High PPS: 0.892]